In [1]:
import numpy as np
import pandas as pd 
from pandas.tseries.holiday import USFederalHolidayCalendar

from utils import load_data, get_train_val_split
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedGroupKFold, train_test_split, TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

from lightgbm import LGBMRegressor
from scipy.stats import kstest, kruskal, mannwhitneyu
from itertools import combinations

In [2]:
data_dict = load_data('ashrae-energy-prediction')

Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.88%
Memory usage of dataframe is 9.60 MB
Memory usage after optimization is: 3.07 MB
Decreased by 68.05%


C:\Users\johns\Desktop\probstats2\EnergyPrediction-ASHRAE\code\utils.py:127: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  weather_train['timestamp'] = pd.to_datetime(weather_train['timestamp'], infer_datetime_format = True, utc = True).astype('datetime64[ns]')


Memory usage of dataframe is 19.04 MB
Memory usage after optimization is: 5.13 MB
Decreased by 73.04%


C:\Users\johns\Desktop\probstats2\EnergyPrediction-ASHRAE\code\utils.py:134: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  train['timestamp'] = pd.to_datetime(train['timestamp'], infer_datetime_format = True, utc = True).astype('datetime64[ns]')


Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 289.19 MB
Decreased by 53.12%
Memory usage of dataframe is 1272.51 MB
Memory usage after optimization is: 358.53 MB
Decreased by 71.82%


C:\Users\johns\Desktop\probstats2\EnergyPrediction-ASHRAE\code\utils.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[(train['site_id'] == 0) & (train['meter'] == 0)]['meter_reading'] = 0.2931 * train[(train['site_id'] == 0) & (train['meter'] == 0)]['meter_reading']


In [3]:
# Add weather features 
weather_features = ['cloud_coverage', 'dew_temperature', 'air_temperature', 
                    'sea_level_pressure', 'wind_direction', 'wind_speed', 'precip_depth_1_hr',]

hourly_by_site = data_dict["X_train"].groupby(['hour', 'month', 'site_id'])[weather_features].mean().reset_index()

data_dict["X_train"] = data_dict["X_train"].merge(
    hourly_by_site, 
    on=['hour', 'month', 'site_id'], 
    how='left', 
    suffixes=(None, '_hourly_by_site')
)

del hourly_by_site

for feature in weather_features:
    # Fill in NA values from weather with hourly by site columns 
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature + "_hourly_by_site"],
        inplace=True
    )
    
    # Fill in the rest with the median 
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature].median(),
        inplace=True
    )
    
    data_dict["X_train"][feature + "_diff_hourly_from_mean"] = data_dict["X_train"][feature] - \
        data_dict["X_train"][feature + "_hourly_by_site"]
    
data_dict["X_train"] = data_dict["X_train"].drop(columns = [feat + "_hourly_by_site" for feat in weather_features])

In [4]:
# Fill in NA with median values for floor count and year_built
for feature in ['year_built', 'floor_count']:
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature].median(), 
        inplace=True
    )

## Examine Differences (Non-Parametric)
Using Bonferonni's Correction

### Milestone 2. 
- Show difference in sites across meter readings 
- get average meter reading per day per site 
- conduct a (non-parametric ANOVA) KS OR pairs (mann-whitney) to show that they are diff 
- train a model per site id (with rudimentary hyperparameter tuning) 
- John sites 0-7, Sharad sites 8-15 

### Milestone 3. Determine, per site, which primary uses are similar (if they have only a few buildings), which are diff
- for a given primary use, if diff, identify "clusters" of buildings that are similar 

In [5]:
total_tests = 4 
alpha = 0.05 / total_tests

for meter in range(0, 4):
    # Perform Kruskal-Wallis between meter readings for all sites 
    readings = []
    for site_id in data_dict['X_train']['site_id'].unique():
        data = data_dict['y_train'][
            (data_dict['X_train']['site_id'] == site_id) & 
            (data_dict['X_train']['meter'] == meter)
        ].dropna()
        
        if len(data) > 0:
            readings.append(data)
            
        
    _, pval = kruskal(*readings)
    print(f"Meter {meter}, pval {pval}")

Meter 0, pval 0.0
Meter 1, pval 0.0
Meter 2, pval 0.0
Meter 3, pval 0.0


In [8]:
total_tests = 0
alpha = 0.05 

for meter in range(0, 4):
    # Perform Mann-Whitney U between meter readings for all sites 
    readings = []
    
    for comb in combinations(data_dict['X_train']['site_id'].unique(), 2):
        try:
            s1 = data_dict['y_train'][
                (data_dict['X_train']['site_id'] == comb[0]) & 
                (data_dict['X_train']['meter'] == meter)
            ].dropna()
            
            s2 = data_dict['y_train'][
                (data_dict['X_train']['site_id'] == comb[1]) & 
                (data_dict['X_train']['meter'] == meter)
            ].dropna()
            
            _, pval = mannwhitneyu(s1, s2)
            print(f"Meter {meter}, site {comb[0]} and {comb[1]}, pval {pval}")
            total_tests += 1 

        except: 
            continue 
            
print(f"Bonferonni Threshold: {alpha / total_tests}")

Meter 0, site 1 and 2, pval 0.0
Meter 0, site 1 and 3, pval 0.0
Meter 0, site 1 and 4, pval 0.0
Meter 0, site 1 and 5, pval 0.0
Meter 0, site 1 and 6, pval 2.3887727765515473e-43
Meter 0, site 1 and 7, pval 0.0
Meter 0, site 1 and 8, pval 0.0
Meter 0, site 1 and 9, pval 3.201351936779842e-49
Meter 0, site 1 and 10, pval 0.0
Meter 0, site 1 and 11, pval 0.0
Meter 0, site 1 and 12, pval 0.0
Meter 0, site 1 and 13, pval 0.0
Meter 0, site 1 and 14, pval 0.0
Meter 0, site 1 and 15, pval 0.0
Meter 0, site 1 and 0, pval 0.0
Meter 0, site 2 and 3, pval 0.0
Meter 0, site 2 and 4, pval 0.0
Meter 0, site 2 and 5, pval 0.0
Meter 0, site 2 and 6, pval 0.0
Meter 0, site 2 and 7, pval 0.0
Meter 0, site 2 and 8, pval 0.0
Meter 0, site 2 and 9, pval 0.0
Meter 0, site 2 and 10, pval 0.0
Meter 0, site 2 and 11, pval 0.0
Meter 0, site 2 and 12, pval 0.0
Meter 0, site 2 and 13, pval 0.0
Meter 0, site 2 and 14, pval 0.0
Meter 0, site 2 and 15, pval 0.0
Meter 0, site 2 and 0, pval 0.0
Meter 0, site 3 and 4, 

In [ ]:
# total_tests = 0

# for site_id in data_dict["X_train"]['site_id'].unique():
#     for primary_use in data_dict["X_train"]['primary_use'].unique():
#         print(f"Site {site_id}, Primary Use {primary_use}")
        
#         subset = data_dict["X_train"][
#             (data_dict["X_train"]['site_id'] == site_id) &
#             (data_dict["X_train"]['primary_use'] == primary_use)
#         ]
        
#         buildings = subset['building_id'].unique()
        
        
#         for meter in range(0, 3):
#             for comb in combinations(buildings, 2):
                
#                 try:
#                     pval, _ = kstest(
#                         subset[(subset['building_id'] == comb[0]) & subset['meter'] == meter]['meter'],
#                         subset[(subset['building_id'] == comb[1]) & subset['meter'] == meter]['meter']
#                     )
        
#                     print(f"\tBuildings {comb[0]} and {comb[1]}, P-value {pval}")
#                     total_tests += 1             
                
#                 except:
#                     continue 


Site 1, Primary Use Education
	Buildings 105 and 106, P-value 0.025280713719830983
	Buildings 105 and 107, P-value 0.0
	Buildings 105 and 108, P-value 0.0
	Buildings 105 and 109, P-value 0.025292628323419608
	Buildings 105 and 110, P-value 0.0
	Buildings 105 and 111, P-value 0.0
	Buildings 105 and 112, P-value 0.025277735127404277
	Buildings 105 and 113, P-value 0.025280713719830983
	Buildings 105 and 114, P-value 0.025292628323419608
	Buildings 105 and 115, P-value 0.0
	Buildings 105 and 116, P-value 0.0
	Buildings 105 and 117, P-value 0.025292628323419608
	Buildings 105 and 118, P-value 0.0
	Buildings 105 and 119, P-value 0.025292628323419608
	Buildings 105 and 120, P-value 0.0
	Buildings 105 and 121, P-value 0.02525092884797553
	Buildings 105 and 122, P-value 0.0
	Buildings 105 and 123, P-value 0.0
	Buildings 105 and 124, P-value 0.0
	Buildings 105 and 125, P-value 0.0
	Buildings 105 and 126, P-value 0.0
	Buildings 106 and 107, P-value 0.025280713719830983
	Buildings 106 and 108, P-

	Buildings 116 and 117, P-value 0.025292628323419608
	Buildings 116 and 118, P-value 0.0
	Buildings 116 and 119, P-value 0.025292628323419608
	Buildings 116 and 120, P-value 0.0
	Buildings 116 and 121, P-value 0.02525092884797553
	Buildings 116 and 122, P-value 0.0
	Buildings 116 and 123, P-value 0.0
	Buildings 116 and 124, P-value 0.0
	Buildings 116 and 125, P-value 0.0
	Buildings 116 and 126, P-value 0.0
	Buildings 117 and 118, P-value 0.025292628323419608
	Buildings 117 and 119, P-value 0.0
	Buildings 117 and 120, P-value 0.025292628323419608
	Buildings 117 and 121, P-value 4.169947544407915e-05
	Buildings 117 and 122, P-value 0.025292628323419608
	Buildings 117 and 123, P-value 0.025292628323419608
	Buildings 117 and 124, P-value 0.025292628323419608
	Buildings 117 and 125, P-value 0.025292628323419608
	Buildings 117 and 126, P-value 0.025292628323419608
	Buildings 118 and 119, P-value 0.025292628323419608
	Buildings 118 and 120, P-value 0.0
	Buildings 118 and 121, P-value 0.025250

	Buildings 145 and 153, P-value 0.05228888021255518
	Buildings 145 and 154, P-value 0.05228888021255518
	Buildings 145 and 155, P-value 0.05228888021255518
	Buildings 146 and 147, P-value 0.0
	Buildings 146 and 148, P-value 0.0
	Buildings 146 and 149, P-value 0.0
	Buildings 146 and 150, P-value 0.0
	Buildings 146 and 151, P-value 0.0
	Buildings 146 and 152, P-value 0.0
	Buildings 146 and 153, P-value 0.0
	Buildings 146 and 154, P-value 0.0
	Buildings 146 and 155, P-value 0.0
	Buildings 147 and 148, P-value 0.0
	Buildings 147 and 149, P-value 0.0
	Buildings 147 and 150, P-value 0.0
	Buildings 147 and 151, P-value 0.0
	Buildings 147 and 152, P-value 0.0
	Buildings 147 and 153, P-value 0.0
	Buildings 147 and 154, P-value 0.0
	Buildings 147 and 155, P-value 0.0
	Buildings 148 and 149, P-value 0.0
	Buildings 148 and 150, P-value 0.0
	Buildings 148 and 151, P-value 0.0
	Buildings 148 and 152, P-value 0.0
	Buildings 148 and 153, P-value 0.0
	Buildings 148 and 154, P-value 0.0
	Buildings 148 a

	Buildings 161 and 277, P-value 0.0
	Buildings 161 and 279, P-value 0.005526707228549865
	Buildings 162 and 163, P-value 0.005526534042098574
	Buildings 162 and 168, P-value 0.005515761423560694
	Buildings 162 and 169, P-value 0.005494850710183075
	Buildings 162 and 173, P-value 3.409961041866971e-07
	Buildings 162 and 174, P-value 3.409961041866971e-07
	Buildings 162 and 175, P-value 0.005526534042098574
	Buildings 162 and 176, P-value 0.005526534042098574
	Buildings 162 and 180, P-value 3.409961041866971e-07
	Buildings 162 and 181, P-value 3.409961041866971e-07
	Buildings 162 and 183, P-value 0.0055256065957645095
	Buildings 162 and 184, P-value 0.005894133035783122
	Buildings 162 and 185, P-value 0.002973789076446076
	Buildings 162 and 187, P-value 0.0055256065957645095
	Buildings 162 and 189, P-value 1.022986679755089e-06
	Buildings 162 and 190, P-value 0.0055256065957645095
	Buildings 162 and 191, P-value 0.0055256065957645095
	Buildings 162 and 192, P-value 0.0054039570174042595


	Buildings 168 and 251, P-value 0.00598436719677975
	Buildings 168 and 252, P-value 0.0055159346100119855
	Buildings 168 and 253, P-value 1.0308894625499221e-05
	Buildings 168 and 255, P-value 1.0772618537879808e-05
	Buildings 168 and 256, P-value 0.00551558823738596
	Buildings 168 and 257, P-value 0.0055159346100119855
	Buildings 168 and 258, P-value 0.0055159346100119855
	Buildings 168 and 259, P-value 0.0055159346100119855
	Buildings 168 and 260, P-value 1.0772618537879808e-05
	Buildings 168 and 261, P-value 0.00598436719677975
	Buildings 168 and 263, P-value 1.0772618537879808e-05
	Buildings 168 and 266, P-value 0.00598436719677975
	Buildings 168 and 271, P-value 0.0055159346100119855
	Buildings 168 and 272, P-value 0.00598436719677975
	Buildings 168 and 277, P-value 0.0055159346100119855
	Buildings 168 and 279, P-value 1.0772618537879808e-05
	Buildings 169 and 173, P-value 0.00549467752400834
	Buildings 169 and 174, P-value 0.00549467752400834
	Buildings 169 and 175, P-value 3.168

...

	Buildings 809 and 843, P-value 0.0
	Buildings 809 and 844, P-value 0.0
	Buildings 809 and 845, P-value 0.0
	Buildings 809 and 846, P-value 0.0
	Buildings 809 and 847, P-value 0.0
	Buildings 809 and 853, P-value 0.0
	Buildings 809 and 854, P-value 0.0
	Buildings 809 and 856, P-value 0.0
	Buildings 809 and 857, P-value 0.0
	Buildings 809 and 869, P-value 0.0
	Buildings 809 and 870, P-value 0.0
	Buildings 809 and 871, P-value 0.0
	Buildings 810 and 812, P-value 0.0
	Buildings 810 and 815, P-value 0.0
	Buildings 810 and 816, P-value 0.0
	Buildings 810 and 817, P-value 0.0
	Buildings 810 and 820, P-value 0.0
	Buildings 810 and 822, P-value 0.0
	Buildings 810 and 831, P-value 0.0
	Buildings 810 and 835, P-value 0.0
	Buildings 810 and 843, P-value 0.0
	Buildings 810 and 844, P-value 0.0
	Buildings 810 and 845, P-value 0.0
	Buildings 810 and 846, P-value 0.0
	Buildings 810 and 847, P-value 0.0
	Buildings 810 and 853, P-value 0.0
	Buildings 810 and 854, P-value 0.0
	Buildings 810 and 856, P-va

	Buildings 811 and 825, P-value 0.0
	Buildings 811 and 826, P-value 0.0
	Buildings 811 and 832, P-value 0.0
	Buildings 811 and 833, P-value 0.0
	Buildings 811 and 834, P-value 0.0
	Buildings 811 and 836, P-value 0.0
	Buildings 811 and 837, P-value 0.0
	Buildings 811 and 838, P-value 0.0
	Buildings 811 and 842, P-value 0.0
	Buildings 811 and 849, P-value 0.0
	Buildings 811 and 850, P-value 0.0
	Buildings 811 and 851, P-value 0.0
	Buildings 811 and 852, P-value 0.0
	Buildings 811 and 859, P-value 0.0
	Buildings 811 and 863, P-value 0.0
	Buildings 811 and 864, P-value 0.0
	Buildings 811 and 865, P-value 0.0
	Buildings 811 and 866, P-value 0.0
	Buildings 811 and 867, P-value 0.0
	Buildings 811 and 868, P-value 0.0
	Buildings 811 and 872, P-value 0.0
	Buildings 813 and 814, P-value 0.0
	Buildings 813 and 818, P-value 0.0
	Buildings 813 and 819, P-value 0.0
	Buildings 813 and 823, P-value 0.0
	Buildings 813 and 824, P-value 0.0
	Buildings 813 and 825, P-value 0.0
	Buildings 813 and 826, P-va

	Buildings 833 and 863, P-value 0.0
	Buildings 833 and 864, P-value 0.0
	Buildings 833 and 865, P-value 0.0
	Buildings 833 and 866, P-value 0.0
	Buildings 833 and 867, P-value 0.0
	Buildings 833 and 868, P-value 0.0
	Buildings 833 and 872, P-value 0.0
	Buildings 834 and 836, P-value 0.0
	Buildings 834 and 837, P-value 0.0
	Buildings 834 and 838, P-value 0.0
	Buildings 834 and 842, P-value 0.0
	Buildings 834 and 849, P-value 0.0
	Buildings 834 and 850, P-value 0.0
	Buildings 834 and 851, P-value 0.0
	Buildings 834 and 852, P-value 0.0
	Buildings 834 and 859, P-value 0.0
	Buildings 834 and 863, P-value 0.0
	Buildings 834 and 864, P-value 0.0
	Buildings 834 and 865, P-value 0.0
	Buildings 834 and 866, P-value 0.0
	Buildings 834 and 867, P-value 0.0
	Buildings 834 and 868, P-value 0.0
	Buildings 834 and 872, P-value 0.0
	Buildings 836 and 837, P-value 0.0
	Buildings 836 and 838, P-value 0.0
	Buildings 836 and 842, P-value 0.0
	Buildings 836 and 849, P-value 0.0
	Buildings 836 and 850, P-va

	Buildings 876 and 895, P-value 0.0021649538672929847
	Buildings 876 and 896, P-value 0.002166444585551819
	Buildings 876 and 897, P-value 0.0
	Buildings 876 and 899, P-value 0.0021656992249224905
	Buildings 876 and 900, P-value 0.0021684322285622226
	Buildings 876 and 901, P-value 0.002165450772045996
	Buildings 876 and 902, P-value 0.0
	Buildings 876 and 903, P-value 0.0021681837720193275
	Buildings 876 and 904, P-value 0.0
	Buildings 876 and 908, P-value 0.0021684322285622226
	Buildings 876 and 911, P-value 0.002166444585551819
	Buildings 876 and 912, P-value 0.002166444585551819
	Buildings 876 and 914, P-value 0.002166196131675402
	Buildings 876 and 915, P-value 0.0021656992249224905
	Buildings 876 and 916, P-value 0.002165947678132274
	Buildings 876 and 917, P-value 0.002165947678132274
	Buildings 876 and 918, P-value 0.0021656992249224905
	Buildings 876 and 919, P-value 0.0
	Buildings 876 and 920, P-value 0.0021684322285622226
	Buildings 876 and 922, P-value 0.002166444585551819


	Buildings 880 and 966, P-value 2.7330036397321678e-06
	Buildings 880 and 968, P-value 1.9876430104037723e-06
	Buildings 880 and 969, P-value 2.4845654289507024e-07
	Buildings 880 and 972, P-value 2.236096886820693e-06
	Buildings 880 and 974, P-value 2.236096886820693e-06
	Buildings 880 and 976, P-value 2.4845654289507024e-07
	Buildings 880 and 978, P-value 0.0
	Buildings 880 and 981, P-value 3.229909059376812e-06
	Buildings 880 and 987, P-value 2.9814565162267215e-06
	Buildings 880 and 989, P-value 7.453686287628436e-07
	Buildings 880 and 991, P-value 1.9876430104037723e-06
	Buildings 880 and 993, P-value 4.969127525011885e-07
	Buildings 880 and 994, P-value 1.9876430104037723e-06
	Buildings 880 and 995, P-value 1.9876430104037723e-06
	Buildings 880 and 996, P-value 0.0
	Buildings 882 and 883, P-value 0.002166444585551819
	Buildings 882 and 884, P-value 0.0
	Buildings 882 and 886, P-value 0.002165947678132274
	Buildings 882 and 889, P-value 0.002165947678132274
	Buildings 882 and 890,